<a href="https://colab.research.google.com/github/descartesmbogning/Marburg-virus-research/blob/main/3_analysis_number_of_institution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
from functools import reduce
import seaborn as sns
import missingno as msno
%matplotlib inline

import matplotlib.pyplot as plt
from scipy.stats import norm


In [ ]:
# Set the figure size - handy for larger output
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = [10, 6]
# Set up with a higher resolution screen (useful on Mac)
%config InlineBackend.figure_format = 'retina'

In [ ]:
from scipy import stats #the only import you actually need for this
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}

In [ ]:
# function that tests distribution then picks test accordingly
def test_sig(x, y):
    if stats.shapiro(x).pvalue >= 0.05 and stats.shapiro(y).pvalue >= 0.05: #if they are normally distriuted
        print("t-test")
        return stats.ttest_ind(x,y)
    else:
        print("mann-whitney")
        return stats.mannwhitneyu(x, y)

## import data

In [ ]:
data = pd.read_csv('D:\MARBURG VIRUS DISEASES\TABLES/data_fund_region_country_and_org_clean_dummies_col_replace_grp.csv')#.drop(['Addresses'], axis=1)
# data = data.loc[data['funding_yes'] == 1]
data.head(2)

,Unnamed: 0,wos_ID,funding_yes,open_access_yes,Funding Name Preferred,europe and central asia,north america,africa,sub-saharan africa,east asia and pacific,...,fulbright foundation - usa,world bank - multilateral,china association for science and technology - china,zhejiang university - china,Funding Orgs,Open Access Designations,"Times Cited, All Databases",180 Day Usage Count,Since 2013 Usage Count,Publication Year
0,0,wos:000904661800001,0,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"green accepted, gold",NaN,14.0,14.0,2023.0
1,1,wos:000921279800001,1,1,National Institutes of Health (NIH)(United Sta...,NaN,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,national institutes of health (nih) [u19ai1427...,hybrid,NaN,NaN,NaN,2023.0


In [ ]:
country_data = pd.read_pickle('D:\MARBURG VIRUS DISEASES\DATA_PREPROCESSING/countries_split.pkl').drop('Addresses', axis=1)
country_data.head(1)

,Afghanistan,United States,Angola,Argentina,Australia,Austria,Bangladesh,Belgium,Belize,Brazil,...,Thailand,Turkey,United Arab Emirates,Uganda,Ukraine,Venezuela,Vietnam,Zambia,Zimbabwe,Sudan
UT (Unique WOS ID),,,,,,,,,,,,,,,,,,,,,
WOS:000904661800001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data_prep = pd.read_excel(f'D:\MARBURG VIRUS DISEASES/data/WOSCC/woscc_bon_23_02_verif_on_wos.xls')
data_prep = data_prep.rename(columns={"UT (Unique WOS ID)": "wos_ID"})
data_prep = data_prep.applymap(lambda s:s.lower() if type(s) == str else s)
#data_prep = data_prep.drop_duplicates('wos_ID')
data_prep.head(1)

,Publication Type,Authors,Book Authors,Book Editors,Book Group Authors,Author Full Names,Book Author Full Names,Group Authors,Article Title,Source Title,...,Web of Science Index,Research Areas,IDS Number,Pubmed Id,Open Access Designations,Highly Cited Status,Hot Paper Status,Date of Export,wos_ID,Web of Science Record
0,j,"bi, jh; wang, hj; han, qx; pei, hy; wang, hl; ...",NaN,NaN,NaN,"bi, jinhao; wang, haojie; han, qiuxue; pei, ho...",NaN,NaN,a rabies virus-vectored vaccine expressing two...,emerging microbes & infections,...,science citation index expanded (sci-expanded),immunology; infectious diseases; microbiology,7j5zq,36453198.0,"green accepted, gold",NaN,NaN,2023-02-22,wos:000904661800001,0


# Extract number of institution

## preprocessing data

In [ ]:
# number of institutions
data_prep['number_of_institution'] =  pd.DataFrame(data_prep['Addresses'].str.split(';', expand=True)).count(axis='columns')
number_institution = data_prep[['wos_ID','number_of_institution']]
# number_institution = number_institution.reset_index().rename(columns = {'UT (Unique WOS ID)':'wos_ID'})
number_institution['wos_ID'] = number_institution['wos_ID'].str.lower()
number_institution

C:\Users\dmbog\AppData\Local\Temp\ipykernel_15036\578369217.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  number_institution['wos_ID'] = number_institution['wos_ID'].str.lower()


KeyError: 'funding_yes'

In [ ]:
number_institution.describe()

,number_of_institution
count,932.000000
mean,8.126609
std,7.964817
min,0.000000
25%,2.000000
50%,6.000000
75%,12.000000
max,82.000000


In [ ]:
number_institution.isna().sum()

wos_ID                   0
number_of_institution    0
dtype: int64

In [ ]:
# number_institution.head(60)

In [ ]:
data_and_number_institution = pd.concat([data.set_index('wos_ID'),
                           number_institution.set_index('wos_ID')],
                          axis=1)
data_and_number_institution = data_and_number_institution.loc[data_and_number_institution['funding_yes'] == 1]
data_and_number_institution.head(2)

,Unnamed: 0,funding_yes,open_access_yes,Funding Name Preferred,europe and central asia,north america,africa,sub-saharan africa,east asia and pacific,/,...,world bank - multilateral,china association for science and technology - china,zhejiang university - china,Funding Orgs,Open Access Designations,"Times Cited, All Databases",180 Day Usage Count,Since 2013 Usage Count,Publication Year,number_of_institution
wos_ID,,,,,,,,,,,,,,,,,,,,,
wos:000921279800001,1,1,1,National Institutes of Health (NIH)(United Sta...,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,national institutes of health (nih) [u19ai1427...,hybrid,NaN,NaN,NaN,2023.0,15
wos:000909001200001,2,1,1,Russian Science Foundation(Russian Science Fou...,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,russian science foundation; [22-13-00036],"gold, green accepted",NaN,3.0,3.0,2023.0,1


In [ ]:
# to drop rows with 0 values in column 'number_of_institution':
data_and_number_institution = data_and_number_institution[data_and_number_institution['number_of_institution'] != 0]

In [ ]:
# data_and_number_institution.describe()

In [ ]:
data_and_number_institution.isna().sum()

Unnamed: 0                      0
funding_yes                     0
open_access_yes                 0
Funding Name Preferred          2
europe and central asia       452
                             ... 
Times Cited, All Databases     30
180 Day Usage Count           341
Since 2013 Usage Count         17
Publication Year                4
number_of_institution           0
Length: 521, dtype: int64

## analysis based on number of country

### proportion all

In [ ]:
data_and_number_institution['number_of_institution'].isna().sum()

0

In [ ]:
data_and_number_institutionT = data_and_number_institution.T

data_and_number_institutionT['countries'] = data_and_number_institutionT.index

number_of_institution_mean_list = []
number_of_institution_count_list = []
for country in data_and_number_institution.columns:
    number_of_institution_count_list.append(data_and_number_institution.dropna(subset=[country])['number_of_institution'].count())
    number_of_institution_mean_list.append(data_and_number_institution.dropna(subset=[country])['number_of_institution'].mean())

# rename countries names on columns
countries = data_and_number_institution.columns.to_list()
number_of_institution_count  = number_of_institution_count_list
number_of_institution_mean = number_of_institution_mean_list

data_and_number_institutionT_count = pd.DataFrame({'countries': countries, 'all_number_of_institution_count': number_of_institution_count})
data_and_number_institutionT_mean = pd.DataFrame({'countries': countries, 'all_number_of_institution_mean': number_of_institution_mean})

# ********
data_and_number_institutionT_concat = data_and_number_institutionT.reset_index().merge(data_and_number_institutionT_count, on = 'countries', how='left')
data_and_number_institutionT_concat = data_and_number_institutionT_concat.merge(data_and_number_institutionT_mean, on = 'countries', how='left')

data_and_number_institutionT_concat_all = data_and_number_institutionT_concat[['countries','all_number_of_institution_count','all_number_of_institution_mean']]

# save data
data_and_number_institutionT_concat_all.to_csv(f'../TABLES/region_countries_org_number_of_institution_count_mean.csv')
data_and_number_institutionT_concat_all

,countries,all_number_of_institution_count,all_number_of_institution_mean
0,Unnamed: 0,577,11.064125
1,funding_yes,577,11.064125
2,open_access_yes,577,11.064125
3,Funding Name Preferred,575,11.069565
4,europe and central asia,125,10.688000
...,...,...,...
516,"Times Cited, All Databases",547,11.065814
517,180 Day Usage Count,236,12.453390
518,Since 2013 Usage Count,560,11.067857
519,Publication Year,573,11.082024


In [ ]:
len(number_of_institution_count)

521

In [ ]:
len(number_of_institution_mean)

521

In [ ]:
len(countries)

521

In [ ]:
vvvvvvvvvvvvvvv

NameError: name 'vvvvvvvvvvvvvvv' is not defined

### proportion fund

In [ ]:
data_and_number_institution_fund = data_and_number_institution.loc[data_and_number_institution['funding_yes'] == 1]
data_and_number_institution_fund.shape

In [ ]:
data_and_number_institutionT = data_and_number_institution_fund.T

data_and_number_institutionT['countries'] = data_and_number_institutionT.index

number_of_institution_mean_list = []
number_of_institution_count_list = []
for country in data_and_number_institution_fund.columns:
    number_of_institution_count_list.append(data_and_number_institution_fund.dropna(subset=[country])['number_of_institution'].count())
    number_of_institution_mean_list.append(data_and_number_institution_fund.dropna(subset=[country])['number_of_institution'].mean())

# rename countries names on columns
countries = data_and_number_institution_fund.columns.to_list()
number_of_institution_count  = number_of_institution_count_list
number_of_institution_mean = number_of_institution_mean_list

data_and_number_institutionT_count = pd.DataFrame({'countries': countries, 'fund_number_of_institution_count': number_of_institution_count})
data_and_number_institutionT_mean = pd.DataFrame({'countries': countries, 'fund_number_of_institution_mean': number_of_institution_mean})

# ********
data_and_number_institutionT_concat = data_and_number_institutionT.reset_index().merge(data_and_number_institutionT_count, on = 'countries', how='left')
data_and_number_institutionT_concat = data_and_number_institutionT_concat.merge(data_and_number_institutionT_mean, on = 'countries', how='left')

data_and_number_institutionT_concat_number_of_institution_fund = data_and_number_institutionT_concat[['countries','fund_number_of_institution_count','fund_number_of_institution_mean']]

# save data
# data_and_number_institutionT_concat_all.to_csv(f'../TABLES/countries_fund_number_of_institution_count_mean.csv')
data_and_number_institutionT_concat_number_of_institution_fund

### proportion no fund

In [ ]:
data_and_number_institution_no_fund = data_and_number_institution.loc[data_and_number_institution['funding_yes'] == 0]
data_and_number_institution_no_fund.shape

In [ ]:
data_and_number_institutionT = data_and_number_institution_no_fund.T

data_and_number_institutionT['countries'] = data_and_number_institutionT.index

number_of_institution_mean_list = []
number_of_institution_count_list = []
for country in data_and_number_institution_no_fund.columns:
    number_of_institution_count_list.append(data_and_number_institution_no_fund.dropna(subset=[country])['number_of_institution'].count())
    number_of_institution_mean_list.append(data_and_number_institution_no_fund.dropna(subset=[country])['number_of_institution'].mean())

# rename countries names on columns
countries = data_and_number_institution_no_fund.columns.to_list()
number_of_institution_count  = number_of_institution_count_list
number_of_institution_mean = number_of_institution_mean_list

data_and_number_institutionT_count = pd.DataFrame({'countries': countries, 'no_fund_number_of_institution_count': number_of_institution_count})
data_and_number_institutionT_mean = pd.DataFrame({'countries': countries, 'no_fund_number_of_institution_mean': number_of_institution_mean})

# ********
data_and_number_institutionT_concat = data_and_number_institutionT.reset_index().merge(data_and_number_institutionT_count, on = 'countries', how='left')
data_and_number_institutionT_concat = data_and_number_institutionT_concat.merge(data_and_number_institutionT_mean, on = 'countries', how='left')

data_and_number_institutionT_concat_number_of_institution_nofund = data_and_number_institutionT_concat[['countries','no_fund_number_of_institution_count','no_fund_number_of_institution_mean']]

# save data
# data_and_number_institutionT_concat_all.to_csv(f'../TABLES/countries_no_fund_number_of_institution_count_mean.csv')
data_and_number_institutionT_concat_number_of_institution_nofund

### concat and save data

In [ ]:
# concatenate dataframes on index
concat_data_number_of_institution_list = pd.concat([data_and_number_institutionT_concat_all.set_index('countries'),
                                    data_and_number_institutionT_concat_number_of_institution_fund.set_index('countries'),
                                    data_and_number_institutionT_concat_number_of_institution_nofund.set_index('countries')],
                                    axis=1)
# save data
concat_data_number_of_institution_list.to_csv(f'../TABLES/concat_data_number_of_institution_list.csv')
concat_data_number_of_institution_list

### significance

In [ ]:
number_institution.describe()

In [ ]:
fund_number_of_institution = data_and_number_institution.loc[data_and_number_institution['funding_yes'] == 1]['number_of_institution']#.to_list()
fund_number_of_institution.describe()

In [ ]:
no_fund_number_of_institution = data_and_number_institution.loc[data_and_number_institution['funding_yes'] == 0]['number_of_institution']#.to_list()
no_fund_number_of_institution.describe()

In [ ]:
test_sig(fund_number_of_institution, no_fund_number_of_institution)

In [ ]:
fund_number_of_institution_new = [x for x in fund_number_of_institution if x != 0]
no_fund_number_of_institution_new = [x for x in no_fund_number_of_institution if x != 0]

In [ ]:
test_sig(fund_number_of_institution_new, no_fund_number_of_institution_new)

In [ ]:
import plotly.express as px
fig = px.box(data_and_number_institution, x="funding_yes", y='number_of_institution', color="funding_yes")
# setting the y-axis range from 0 to 50
fig.update_layout(yaxis_range=[-2,100])
fig.update_layout(font_size=20)

fig.show()